# WaterLily-RL Tutorial - Getting Started
WaterLily-RL is a Deep Reinforcement Learning(DRL) simulation framework forcing on the fluid dyamics. 
It trains the agent in a vitual fluid environment created by WaterLily, a simple and fast fluid simulator.

## Introduction
This tutorial provides a basic DRL task in fluid dynamics - a agent learns how to execute a direct force 
to restrain the virbration caused by incoming flow (Vortex-Induced Vibration).

## Install Python Dependencies Using Pip
List of full Python dependencies can be found in the setup.py,follow the instruction in README can help you install them.

## Install Julia Dependencies
List of full Julia dependencies can be found in the README.

## Multi-threads
If your operating system is Linux, you can build multiply threads to accelerate the simulation. 
It is notrecommanded to implement multi-threads in Windows, since it may result in issues in rendering.

In [ ]:
import os
os.environ["JULIA_NUM_THREADS"] = "8" # build 8 threads
from julia import Julia
jl = Julia(compiled_modules=False)
from julia import Main
print(Main.eval("Threads.nthreads()"))


## Imports
As we used Stable-Baselines3 as the benchmark DRL library, 'gymnasium' and 'stable_baselines3' 
are required. You can import the PPO here as the whole tutorial is based on it.

In [ ]:


"""

Lib 支持

"""
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import BaseCallback, CheckpointCallback, CallbackList
from src.VIV_gym import JuliaEnv


## Return the episodic reward and Set the checkpoint

In order to deal with the interrupt during learning processing, or willing to continuing 
the completed training, you need to set the checkpoints and callback later.

In [ ]:
"""

反馈reward和建立checkpoint

"""
class RewardLoggerCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.episode_rewards = []
        self.current_rewards = None
        self.episode_steps = []          # 存每个 episode 的 step 数
        self.current_steps = None

    def _on_training_start(self) -> None:
        self.current_rewards = np.zeros(self.training_env.num_envs)
        self.current_steps = np.zeros(self.training_env.num_envs, dtype=int)

    def _on_step(self) -> bool:
        rewards = self.locals["rewards"]
        dones = self.locals["dones"]
        self.current_rewards += rewards
        self.current_steps += 1   # 每个 step 累加


        for i, done in enumerate(dones):
            if done:
                self.episode_rewards.append(self.current_rewards[i])
                self.episode_steps.append(self.current_steps[i])  # 记录步数

                print(f"Episode finished after {self.current_steps[i]} steps")
                print(f"Episode reward: {self.current_rewards[i]:.2f}")
                # reset
                self.current_rewards[i] = 0.0
                self.current_steps[i] = 0

        return True

checkpoint_callback = CheckpointCallback(
    save_freq= 10000,
    save_path="./checkpoints/",
    name_prefix="ppo_model",
    save_replay_buffer=True,
    save_vecnormalize=True
)

## Parameters
The three dict "statics", "variables"and "spaces" are essential parameters which should be
determined before running. 'VIV_gym' from 'src' floder is the source code to refer the julia
file.

In [ ]:
"""

训练用参数(VIV)

"""
diameter = 16
def pos_generator():
    return [0.0, np.random.uniform(- diameter/6, diameter/6)]

# static parameters
statics = {
    "L_unit": diameter,                         # dimension of the object (diameter of the circle)
    "action_scale": 50,                         # amplifing the action from [-1,1] to [-50,50]
    "size": [10, 8],                            # size ratio of the simulation env
    "location": [3, 4]                          # location of the object in the simulation env (size ratio)
}
#variable parameters
variables = {
    "position":[0.0, diameter/6],               # manual position of the object respects to the initial location
    "velocity":[0.0, 0.0]                       # initial velocity
}
# size of action sapce and observation spaces
spaces = {
    "action":1,                                 # action space
    "observation":3                             # observation spaces
}

from src.VIV_gym import VIVEnv

## Create the env and instantiate the agent
### Refer the WaterLily env and build the model
For this example, we will use the basic VIV senario as the simulation env. Thus,
we can set the 'VIVEnv' to 'env' and import the previously determined parameters
(statics, variables and spaces). It is recommanded to set the 'render_mode' as 
'None', since create image will extremely slow down the train process.

Then, as mentioned before, we accepct PPO algorthm and the policy is "MlpPolicy" 
since the input action is a vector.

In [ ]:
# Build the env and model
env = DummyVecEnv([lambda: JuliaEnv(render_mode=None, env = VIVEnv, max_episode_steps=2000, statics = statics, 
                                    variables = variables, spaces = spaces, verbose=1)])

model = PPO(
    "MlpPolicy",
    env=env,
    verbose=1,
    device = 'cpu'
)

let's define the callback function from previously determined checkpoint function.

In [ ]:
# set the checkpoint and reward sum in callback
reward_callback = RewardLoggerCallback()
callback = CallbackList([checkpoint_callback, reward_callback])

### Train the agent and save it

In [ ]:
# train and save the agent
model.learn(total_timesteps=100_000, callback = callback)
model.save("./model/PPO_model")

In [ ]:
# collect te rewards and close the env
rewards = np.array(reward_callback.episode_rewards)
np.save('rewards.npy', rewards)
env.close()

## Evaluate the train process
### Image
After collecting the rewards, you can draw the rewards during training process here.
Apperantly, the reward doesn't converge and the performance is not well. Don't worry,
we can continuously train it from the last checkpoint later.

In [ ]:
"""

绘图功能

"""
import matplotlib.pyplot as plt
import numpy as np

# load the rewards
rewards = np.load('rewards.npy')
# param：the sliding window for means and std
window = 10

def plot_rewards(rewards, window=100):
    # calculate the means and stds
    def moving_avg(x, w):
        return np.convolve(x, np.ones(w)/w, mode='valid')

    mean = moving_avg(rewards, window)
    std = np.array([
        np.std(rewards[max(0, i - window + 1):i + 1])
        for i in range(window - 1, len(rewards))
    ])

    # x-axis
    x = np.arange(window - 1, len(rewards))

    # plot
    plt.figure(figsize=(12, 6))
    plt.plot(x, mean, label='Mean Reward')
    plt.fill_between(x, mean - std, mean + std, alpha=0.3, label='±1 Std Dev')
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.title("Episode Reward over Training")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_rewards(rewards,window)

### GIF & info
You probably wanna see the trained agent's motion in a grphic windows or a gif of 
the whole moving process. You can create the gif here by play the trained model in
our sim env.

Moreover, if you wanted to analyse the dynamics of the trained agent, you can save
the information, determined in julia files, into a npy file.

In [ ]:
# create gif after training

import numpy as np
from stable_baselines3 import PPO
from src.VIV_gym import JuliaEnv
from src.gif import create_GIF

infos = []

# same simulation env while 'render_mode' is 'rgb_array' to create images
env = JuliaEnv(render_mode="rgb_array", env = VIVEnv, max_episode_steps=2000, statics = statics, variables = variables, spaces = spaces, verbose=True)

# load the trained PPO_model
model = PPO.load("./model/PPO_model", env=env)

# video frame
frames = []

# reset the env
obs, _ = env.reset()

done = False
truncated = False

# if 'not done', then continue to perform the simulation operation based on trained model
while not done and not truncated:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)

# save as gif
input_frame = "images"
output_gif = "./result/train_policy_demo.gif"
create_GIF(input_frame, output_gif)
env.close()

# save the info
np.save("info_PPO.npy", info["info"])


### Analysis
Now you have a npy file which contains all dynamic information, so you can plow a 
line chart of fluid force, agent applied force and the displacement in y direction.
The result is not appropriate, as the train is not 100% converged, you can run the 
callback this time to continue the training.


In [ ]:
import matplotlib.pyplot as plt

info = np.load("info_PPO.npy", allow_pickle=True)
force = [f["F"] for f in info[50:]]
y_force = [f["fluid_force_y"] for f in info[50:]]
x_force = [f["fluid_force_x"] for f in info[50:]]
y_dis = [f["y_dis"] for f in info[50:]]
x_dis = [f["x_dis"] for f in info[50:]]

x = np.arange(len(y_force))
# x2 = np.arange(len(y_dis2))

# 画图
plt.figure(figsize=(8, 5))
plt.plot(x, force, label="y_force", color="red")
plt.plot(x, y_force, label="y_fluid", color="blue")
plt.plot(x, y_dis, label="y_displacement", color="green")

# 图例、标签、标题
plt.xlabel("step")
plt.ylabel("force & displacement")
plt.title("Force and Displacement in y direction")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Callback
You found out that the train doesn't end up with a converged reward. You can
start a subsequent process from the checkpoint of 100k steps. Run the other
200k steps train now:


In [ ]:
"""

加载checkpoint并继续训练

"""
env = DummyVecEnv([lambda: JuliaEnv(render_mode=None, env = VIVEnv, max_episode_steps=2000, statics = statics, 
                                    variables = variables, spaces = spaces, verbose=1)])

reward_callback = RewardLoggerCallback()
callback = CallbackList([checkpoint_callback, reward_callback])

model = PPO.load("./checkpoints/ppo_model_100000_steps", env=env, device='cpu')
model.learn(total_timesteps=200_000, callback = callback)
rewards_ex = np.array(reward_callback.episode_rewards)
rewards = np.load('rewards.npy')
rewards = np.concatenate([rewards, rewards_ex])
np.save('rewards.npy', rewards)
model.save("./model/PPO_model_100k-300k")
env.close()

Plot the means and stds of the rewards now. You can find it is nearly converged.
Apparently, the agent learned a policy to execute the force to counteract the lift 
force, keeps the agent around y=0.

In [ ]:
"""

Reward Image

"""
import matplotlib.pyplot as plt
import numpy as np

# load the rewards
rewards = np.load('rewards.npy')
# param：the sliding window for means and std
window = 10

def plot_rewards(rewards, window=100):
    episode = np.arange(len(rewards))

    # calculate the means and stds
    def moving_avg(x, w):
        return np.convolve(x, np.ones(w)/w, mode='valid')

    mean = moving_avg(rewards, window)
    std = np.array([
        np.std(rewards[max(0, i - window + 1):i + 1])
        for i in range(window - 1, len(rewards))
    ])

    # x-axis
    x = np.arange(window - 1, len(rewards))

    # plot
    plt.figure(figsize=(12, 6))
    plt.plot(x, mean, label='Mean Reward')
    plt.fill_between(x, mean - std, mean + std, alpha=0.3, label='±1 Std Dev')
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.title("Episode Reward over 250k Steps Training")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_rewards(rewards,window)

In [ ]:
"""

GIF

"""
# create gif after training

import numpy as np
from stable_baselines3 import PPO
from src.VIV_gym import JuliaEnv
from src.gif import create_GIF

infos = []

# same simulation env while 'render_mode' is 'rgb_array' to create images
env = JuliaEnv(render_mode="rgb_array", env = VIVEnv, max_episode_steps=2000, statics = statics, variables = variables, spaces = spaces, verbose=True)

# load the trained PPO_model
model = PPO.load("./model/PPO_model_100k-200k", env=env)

# video frame
frames = []

# reset the env
obs, _ = env.reset()

done = False
truncated = False

# if 'not done', then continue to perform the simulation operation based on trained model
while not done and not truncated:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)

# save as gif
input_frame = "images"
output_gif = "./result/train_policy_demo.gif"
create_GIF(input_frame, output_gif)
env.close()

# save the info
np.save("info_PPO.npy", info["info"])


You will find that the agent is completely trained, and the y-displacement are
nearly 0 all time.

In [ ]:
import matplotlib.pyplot as plt

info = np.load("info_PPO.npy", allow_pickle=True)
force = [f["F"] for f in info[50:]]
y_force = [f["fluid_force_y"] for f in info[50:]]
x_force = [f["fluid_force_x"] for f in info[50:]]
y_dis = [f["y_dis"] for f in info[50:]]
x_dis = [f["x_dis"] for f in info[50:]]

x = np.arange(len(y_force))
# x2 = np.arange(len(y_dis2))

# 画图
plt.figure(figsize=(8, 5))
plt.plot(x, force, label="y_force", color="red")
plt.plot(x, y_force, label="y_fluid", color="blue")
plt.plot(x, y_dis, label="y_displacement", color="green")

# 图例、标签、标题
plt.xlabel("step")
plt.ylabel("force & displacement")
plt.title("Force and Displacement in y direction")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()